In [78]:
chapter_dict = {
    '1': 'Basic Financial Calculations',
    '2': 'Corporate Valuation Overview',
    '3': 'Calculating the Weighted Average Cost of Capital (WACC)',
    '4': 'Valuation Based on the Consolidated Statement of Cash Flows',
    '5': 'Pro Forma Financial Statement Modeling',
    '6': 'Building a Pro Forma Model: The Case of Caterpillar',
    '7': 'Financial Analysis of Leasing',
    '8': 'Portfolio Models—Introduction',
    '9': 'Calculating Efficient Portfolios',
    '10': 'Calculating the Variance-Covariance Matrix',
    '11': 'Estimating Betas and the Security Market Line',
    '12': 'Efficient Portfolios Without Short Sales',
    '13': 'The Black-Litterman Approach to Portfolio Optimization',
    '14': 'Event Studies',
    '15': 'Introduction to Options',
    '16': 'The Binomial Option Pricing Model',
    '17': 'The Black-Scholes Model',
    '18': 'Option Greeks',
    '19': 'Real Options',
    '20': 'Duration',
    '21': 'Immunization Strategies',
    '22': 'Modeling the Term Structure',
    '23': 'Calculating Default-Adjusted Expected Bond Returns',
    '24': 'Generating and Using Random Numbers',
    '25': 'An Introduction to Monte Carlo Methods',
    '26': 'Simulating Stock Prices',
    '27': 'Monte Carlo Simulations for Investments',
    '28': 'Value at Risk (VaR)',
    '29': 'Simulating Options and Option Strategies',
    '30': 'Using Monte Carlo Methods for Option Pricing',
    '31': 'Data Tables',
    '32': 'Matrices',
    '33': 'Excel Functions',
    '34': 'Array Functions',
    '35': 'Some Excel Hints',
    '36': 'VISUAL BASIC FOR APPLICATIONS (VBA) - User-Defined Functions with VBA',
    '37': 'VISUAL BASIC FOR APPLICATIONS (VBA) - Variables and Arrays',
    '38': 'VISUAL BASIC FOR APPLICATIONS (VBA) - Subroutines and User Interaction',
    '39': 'VISUAL BASIC FOR APPLICATIONS (VBA) - Objects and Add-Ins'
}


In [158]:
!langchain --version

zsh:1: command not found: langchain


# Langchain

In [102]:
data_dict.keys()

dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39'])

## Creating Vector DB

In [92]:
import json
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
import os
import openai

openai.api_key = "key"

# 1. Load your data_dict
with open("data_dict.json", "r") as f:
    data_dict = json.load(f)

# 2. Create a list of Documents for LangChain
# Each "Document" contains:
# - page_content: the text
# - metadata (optionally, e.g. chapter, section)
documents = []

for chapter, sections in data_dict.items():
    for section_id, text in sections.items():
        doc = Document(
            page_content=text,
            metadata={
                "chapter": chapter,
                "section": section_id
            }
        )
        documents.append(doc)

# 3. Initialize embeddings
embedding_model = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=openai.api_key
)

# 4. Build the FAISS vector store from the documents
vectorstore = FAISS.from_documents(documents, embedding_model)

# 5. Save it locally (LangChain style)
if not os.path.exists("faiss_store"):
    os.makedirs("faiss_store")
vectorstore.save_local("faiss_store")

print("LangChain-compatible FAISS store created and saved in 'faiss_store'.")


LangChain-compatible FAISS store created and saved in 'faiss_store'.


## Testing Vector DB

In [95]:
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# 1. Set your API key
openai.api_key = "key"

# 2. Re-initialize the same embeddings you used earlier
embedding_model = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=openai.api_key
)

# 3. Load the vector store from your saved directory
vectorstore = FAISS.load_local(
    folder_path="faiss_store",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)
# 4. Define a question you want to ask
query = "How does the deposit frequency affect the future value calculation? How can we calculate the future value with annual deposits in Excel?"

# 5. Query the vector store, asking for the top 3 matches
results = vectorstore.similarity_search(query, k=3)

# 6. Print out the matching Documents
for i, doc in enumerate(results, start=1):
    print(f"== Result {i} ==")
    print("Metadata:", doc.metadata)
    print("Content:", doc.page_content[:200], "...")  # Truncate for brevity
    print()


== Result 1 ==
Metadata: {'chapter': '1', 'section': '1.6'}
Content: 1.6 Future Values and Applications

We start with a triviality. Suppose you deposit 1,000 in an account today,
leaving it there for 10 years. Suppose the account draws annual interest of
10%. How much ...

== Result 2 ==
Metadata: {'chapter': '1', 'section': '1.7'}
Content: 1.7 A Pension Problem—Complicating the Future Value Problem

A typical exercise is the following: You are currently 55 years old and intend
to retire at age 60. To make your retirement easier, you int ...

== Result 3 ==
Metadata: {'chapter': '1', 'section': '1.1'}
Content: 1.1 Overview

T his chapter aims to give you some fi nance basics and their Excel implementa-
tion. If you have had a good introductory course in fi nance, this chapter is
likely to be at best a refre ...



## Implmenting RAG

In [97]:
import os
import openai
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

openai.api_key = "key"

# 1. Load embeddings
embedding_model = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=openai.api_key
)

# 2. Load the LangChain-compatible FAISS store
vectorstore = FAISS.load_local(
    folder_path="faiss_store",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

# 3. Create a retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",  # or "similarity_score_threshold", etc.
    search_kwargs={"k": 3}     # top_k passages to retrieve
)

# 4. Create a ChatOpenAI model for generation
llm = ChatOpenAI(
    openai_api_key=openai.api_key,
    temperature=0.0
)

# 5. Build the RetrievalQA chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # or "refine", etc.
    retriever=retriever
)




/var/folders/mj/yn1hkgnn6hd7nwmjt1_x7qz00000gn/T/ipykernel_44700/2480704910.py:30: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [99]:
# 6. Ask a question
query = "How does the deposit frequency affect the future value calculation? How can we calculate the future value with annual deposits in Excel?"
result = rag_chain.run(query)
print("User Query:", query)
print("RAG Answer:", result)

/var/folders/mj/yn1hkgnn6hd7nwmjt1_x7qz00000gn/T/ipykernel_44700/2378721939.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = rag_chain.run(query)


User Query: How does the deposit frequency affect the future value calculation? How can we calculate the future value with annual deposits in Excel?
RAG Answer: The deposit frequency affects the future value calculation by impacting the compounding of interest. More frequent deposits allow for more frequent compounding, which can lead to a higher future value due to the effects of compound interest.

To calculate the future value with annual deposits in Excel, you can use the FV function. The FV function calculates the future value of an investment based on a constant interest rate, regular deposits, and a constant payment amount. Here is an example formula for calculating the future value with annual deposits in Excel:

`=FV(rate, nper, pmt, [pv], [type])`

- `rate`: The interest rate per period.
- `nper`: The total number of payment periods.
- `pmt`: The payment made each period; it must remain constant.
- `[pv]`: Optional. The present value, or the total amount that a series of futu

In [130]:
import os
import openai
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate

openai.api_key = "key"

# 1. Load embeddings
embedding_model = OpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_key=openai.api_key
)

# 2. Load FAISS store
vectorstore = FAISS.load_local(
    folder_path="faiss_store",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)

# 3. Create a retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# 4. Create LLM
llm = ChatOpenAI(
    model_name="gpt-4",   # gpt-4
    openai_api_key=openai.api_key,
    temperature=0.1
)
# 5. Build a custom prompt template
#    Note: For "stuff"-type QA, your template should include `{context}` and `{question}` placeholders
template_text = """

You are a helpful answer generation assistant who can take in a question and generate a comprehensive and detailed answer based on the question provided.

You will be given the question along with context that is extracted from the book "FINANCIAL MODELING" from Simon Benninga. These contexts 
are basically 3 sections from any chapter within the group that may help answering the question better

Important thing to note: The context is just given for reference. You have to generate a comprehensive answer based on your own knowledge.

Below you can find the details:

Question - {question}

Context:
{context}

Important things to note:
The context is just given for reference. You have to generate a detailed answer based on your own knowledge. 
Where-ever applicable keep a good balance between theoretical (formulas, definitions) and practical examples. You can also give examples on how you can do it in excel where-ever necessary.
"""

prompt = PromptTemplate(
    template=template_text,
    input_variables=["context", "question"]
)

# 6. Create the RetrievalQA chain, passing in our custom prompt
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)



In [132]:
# 7. Ask your question
query = "How does the deposit frequency affect the future value calculation? How can we calculate the future value with annual deposits in Excel?"

response = rag_chain.run(query)
print(response)


The deposit frequency can significantly affect the future value calculation. When deposits are made more frequently, such as annually, the future value of the investment will be higher due to the compounding effect. In the context provided, we see how annual deposits of $1,000 each year for 10 years at an interest rate of 10% result in a future value of $17,531.17. This is calculated by considering the initial deposit, subsequent annual deposits, and the interest earned on the account.

To calculate the future value with annual deposits in Excel, you can use the FV function. The formula for calculating the future value with annual deposits in Excel is:

=FV(rate, nper, pmt, [pv], [type])

- rate: The interest rate per period.
- nper: The total number of payment periods.
- pmt: The payment made each period; it should be a negative number.
- pv: (Optional) The present value, or the total amount that a series of future payments is worth now.
- type: (Optional) When payments are due, eithe